### Import libraries

In [ ]:
import chessdotcom
from chessdotcom import ChessDotComClient
import re
import datetime
import pytz
import pandas as pd
pd.set_option('display.max_colwidth', None)

<br>

## To start, run the below cell and enter your Chess.com username (case insensitive) 
>#### If you want to see a Grandmaster's games, enter "magnuscarlsen"

In [ ]:
# Connect to the Chess.com API and check that the username exists
client = ChessDotComClient(user_agent = "My Python Application...")
response = 0
while response == 0:
    username = str(input("Enter Username: "))
    try:
        response = client.get_player_profile(username)
        print('Username entered successfully')
    except chessdotcom.errors.ChessDotComClientError:
        print('Invalid Username')

# Reassigning the username variable to the value stored on chess.com
url = response.player.url
pattern = r'[^/]+$'
match = re.findall(pattern, url)
username = match[0]

<br>

## To get accurate game dates, run the below cell and enter your local timezone

>#### Timezones can be found in the "Timezone List" Excel file. If you do not wish to specify a timezone, enter "UTC"

In [ ]:
# Make list of all acceptable timezone values
tz_list = []
for tz in pytz.all_timezones:
    tz_list.append(tz)

# Check that the user-entered timezone is in the timezone list
while True:
    timezone = str(input("Enter Timezone: "))
    if timezone in tz_list:
        print('Timezone entered successfully')
        break
    else:
        print('Invalid Timezone')

<br>

## Display Player Profile 

In [ ]:
# Pulling profile data from the Chess.com API
if response.player.name is None:
    name = pd.NA
else: 
    name = response.player.name

if response.player.country is None:
    country = pd.NA
else: 
    response_country = client.get_country_details(response.player.country[-2:])
    country = response_country.country.name

if response.player.joined is None:
    joined = pd.NA
else: 
    joined = datetime.datetime.fromtimestamp(response.player.joined)

if response.player.last_online is None:
    last_online = pd.NA
else: 
    last_online = datetime.datetime.fromtimestamp(response.player.last_online)

# Create dataframe to display profile
profile = {
    "Profile_Info": ["Name", "Country", "Joined", "Last Online"],
    username: [name, country, joined, last_online]}

df = pd.DataFrame(profile)
df.set_index("Profile_Info", inplace=True)
df = df.rename_axis(None)
df

<br>

## Display All-Time Player Stats 

>#### Game types displayed include Rapid Chess, Blitz Chess, Bullet Chess, and Daily Chess
>#### "Rating" is how strong a player is according to Chess.com for each game type
>#### "Record" is the number of Wins-Losses-Draws for each game type

In [ ]:
# Pulling stats data from the Chess.com API
response2 = client.get_player_stats(username)

if response2.stats.chess_rapid is None:
    rapid_rating = pd.NA
    rapid_wld = pd.NA
else: 
    rapid_rating = response2.stats.chess_rapid.last.rating
    rapid_wld = f"{str(response2.stats.chess_rapid.record.win)}-{str(response2.stats.chess_rapid.record.loss)}-{str(response2.stats.chess_rapid.record.draw)}"

if response2.stats.chess_blitz is None:
    blitz_rating = pd.NA
    blitz_wld = pd.NA
else: 
    blitz_rating = response2.stats.chess_blitz.last.rating
    blitz_wld = f"{str(response2.stats.chess_blitz.record.win)}-{str(response2.stats.chess_blitz.record.loss)}-{str(response2.stats.chess_blitz.record.draw)}"
    
if response2.stats.chess_bullet is None:
    bullet_rating = pd.NA
    bullet_wld = pd.NA
else: 
    bullet_rating = response2.stats.chess_bullet.last.rating
    bullet_wld = f"{str(response2.stats.chess_bullet.record.win)}-{str(response2.stats.chess_bullet.record.loss)}-{str(response2.stats.chess_bullet.record.draw)}"

if response2.stats.chess_daily is None:
    daily_rating = pd.NA
    daily_wld = pd.NA
else: 
    daily_rating = response2.stats.chess_daily.last.rating
    daily_wld = f"{str(response2.stats.chess_daily.record.win)}-{str(response2.stats.chess_daily.record.loss)}-{str(response2.stats.chess_daily.record.draw)}"

# Create dataframe to display stats
stats = {
    "Profile_Info": ["Rapid Chess Rating", "Rapid Chess Record",
                     "Blitz Chess Rating", "Blitz Chess Record",
                     "Bullet Chess Rating", "Bullet Chess Record",
                     "Daily Chess Rating", "Daily Chess Record"],
    "Username": [rapid_rating, rapid_wld,
                 blitz_rating, blitz_wld,
                 bullet_rating, bullet_wld,
                 daily_rating, daily_wld
                 ]}

df = pd.DataFrame(stats)
df.rename(columns={"Username": username}, inplace=True)
df.set_index("Profile_Info", inplace=True)
df = df.rename_axis(None)
df

<br>

## Create a list of all "live" games the player has completed

>#### This cell may take a few minutes to run if the player has thousands of games
>#### The list will not include games played against a computer, nor games played with non-traditional rules such as Chess 960

In [ ]:
# Create a list of each month that the player has played a game
# This is nessesary because the game data is stored by month in the Chess.com API
response3 = client.get_player_game_archives(username)
months_list = []
for i in range(len(response3.archives)):
    months_list.append(response3.archives[i][-7:])

# Create a list of every live game the player has played via looping through each month
live_games = []
for i in range(len(months_list)):
    year = int(months_list[i][:4])
    month = int(months_list[i][5:])
    response4 = client.get_player_games_by_month(username, year, month)
    for i in range(len(response4.games)):
        if response4.games[i].pgn and (response4.games[i].pgn[:20] == '[Event "Live Chess"]' or response4.games[i].pgn[:20] == '[Event "Let\'s Play"]'):
            live_games.append(response4.games[i])

<br>

### Define a function to convert game dates to the correct timezone

In [ ]:
def convert_to_timezone(date, timezone):
    date = pd.to_datetime(date)
    date = date.tz_localize('UTC')
    date = str(date.tz_convert(timezone))
    return date[:-9]

<br>

### Define a function to retrieve relevant game information

In [ ]:
def game_info(num):
    if live_games[num].white.username != username:
        opponent = live_games[num].white.username
        my_result = live_games[num].black.result
        opponent_result = live_games[num].white.result
        my_rating = int(live_games[num].black.rating)
        opponent_rating = int(live_games[num].white.rating)
        pieces = 'black'
    else: 
        opponent = live_games[num].black.username
        my_result = live_games[num].white.result
        opponent_result = live_games[num].black.result
        my_rating = int(live_games[num].white.rating)
        opponent_rating = int(live_games[num].black.rating)
        pieces = 'white'
    
    if my_result == 'win':
        result = 'win'
        game_ending = opponent_result
    elif my_result in ['resigned', 'checkmated', 'timeout']:
        result = 'loss'
        game_ending = my_result
    else:
        result = 'draw'
        game_ending = my_result

    list = [pieces, opponent, result, game_ending, my_rating, opponent_rating]
    return list

<br>

### Define a function to retrieve and format time controls  

In [ ]:
def time_control(num):
    try:
        time = live_games[num].time_control
        if '+' in time:
            if time[-2] == '+':
                time = str(int(time[:-2])//60) + ' minutes, ' + time[-1] + ' seconds per move'
            else:
                time = str(int(time[:-3])//60) + ' minutes, ' + time[-2:] + ' seconds per move'
        elif live_games[num].time_class == 'daily':
            time = 'daily'
        else:
            time = str(int(time)//60) + ' minutes, ' +  '0 seconds per move'

        if time == '0 minutes, 0 seconds per move':
            time = 'less than 1 minute'
        
        return time
        
    except ValueError:
        time = 'N/A'
        return time

<br>

### Define a function to retrieve the game opening variations

In [ ]:
# Regular Expressions are used to capture the names of the openings because they are stored in a URL
def opening(num):
    try:
        text = live_games[num].pgn
        pattern0 = r'ECOUrl\s*"([^"]+)"'
        match0 = re.findall(pattern0, text)
        pattern1 = r'[^/]+$'
        match1 = re.findall(pattern1, match0[0])
        pattern2 = r'-\d.*$|with-\d.*$|\.{3}.*$'
        match2 = re.sub(pattern2, "", match1[0])
        match3 = re.sub(r'-', " ", match2)
        return match3
    except IndexError:
        match3 = 'N/A'
        return match3

<br>

### Create a dataframe of all games with columns to display the details of each game

In [ ]:
# Initializing a new pandas dataframe
columns = ['Date', 'Played As', 'My Rating', 'Opponent', 'Opponent Rating', 'Result', 'Game Ending', 'Game Type', 'Time Controls', 'Opening', 'URL']
chess_games = pd.DataFrame(columns=columns)

dates = []
played_as = []
my_rating = []
opponent = []
opponent_rating = []
result = []
game_ending = []
game_type = []
controls = []
chess_opening = []
url = []

# This FOR loop calls the functions that we defined earlier
for i in range(len(live_games)):
    readable_date = datetime.datetime.utcfromtimestamp(live_games[i].end_time)
    dates.append(convert_to_timezone(readable_date, timezone))
    played_as.append(game_info(i)[0])
    opponent.append(game_info(i)[1])
    result.append(game_info(i)[2])
    game_ending.append(game_info(i)[3])
    my_rating.append(game_info(i)[4])
    opponent_rating.append(game_info(i)[5])
    game_type.append(live_games[i].time_class)
    controls.append(time_control(i))
    chess_opening.append(opening(i))
    url.append(live_games[i].url)

# Adding the lists that contain the game data to the dataframe columns
chess_games['Date'] = dates
chess_games['Played As'] = played_as
chess_games['My Rating'] = my_rating
chess_games['Opponent'] = opponent
chess_games['Opponent Rating'] = opponent_rating
chess_games['Result'] = result
chess_games['Game Ending'] = game_ending
chess_games['Game Type'] = game_type
chess_games['Time Controls'] = controls
chess_games['Opening'] = chess_opening
chess_games['URL'] = url
chess_games['Year'] = chess_games['Date'].str[:4]
chess_games['Month'] = chess_games['Date'].str[5:7]

<br>

### Define a function to filter the dataframe by any combination of 13 different criteria

In [ ]:
def filtered_results(played_as, my_rating_lowerbound, my_rating_upperbound, opponent, opponent_rating_lowerbound, opponent_rating_upperbound, 
                     year, month, result, game_ending, game_type, time_controls, opening):
    global filtered_df
    filtered_df = chess_games
    if played_as != 'ALL':
        filtered_df = filtered_df[filtered_df['Played As'] == played_as]
    if my_rating_lowerbound != 'ALL':
        filtered_df = filtered_df[filtered_df['My Rating'] >= int(my_rating_lowerbound)]
    if my_rating_upperbound != 'ALL':
        filtered_df = filtered_df[filtered_df['My Rating'] <= int(my_rating_upperbound)]
    if opponent != 'ALL':
        filtered_df = filtered_df[filtered_df['Opponent'].str.contains(opponent, case=False, na=False)]
    if opponent_rating_lowerbound != 'ALL':
        filtered_df = filtered_df[filtered_df['Opponent Rating'] >= int(opponent_rating_lowerbound)]
    if opponent_rating_upperbound != 'ALL':
        filtered_df = filtered_df[filtered_df['Opponent Rating'] <= int(opponent_rating_upperbound)]
    if year != 'ALL':
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if month != 'ALL':
        filtered_df = filtered_df[filtered_df['Month'] == month]
    if result != 'ALL':
        filtered_df = filtered_df[filtered_df['Result'] == result]
    if game_ending != 'ALL':
        filtered_df = filtered_df[filtered_df['Game Ending'] == game_ending]
    if game_type != 'ALL':
        filtered_df = filtered_df[filtered_df['Game Type'] == game_type]
    if time_controls != 'ALL':
        filtered_df = filtered_df[filtered_df['Time Controls'] == time_controls]
    if opening != 'ALL':
        filtered_df = filtered_df[filtered_df['Opening'].str.contains(opening, case=False, na=False)]
    filtered_df = filtered_df.reset_index(drop=True)
    return filtered_df.iloc[:, :-2]

<br>
<br>
<br>

# Run the below cell to print the dataframe of all games you've played.
>#### You can filter the dataframe by editing the strings assigned to each variable and then running the cell again
>#### For the "opponent" and "opening" variables, there is a built-in wildcard search so you don't have to enter an exact match
>#### For the "rating lower/upperbound" variables, enter your number as a string, do not assign the variable to be an integer
>#### If the dataframe has no rows, either there is no data for your filtering criteria, or you entered a misspelled value
>#### To un-filter a variable, simply change the string back to 'ALL' and re-run the cell

In [ ]:
# Edit the below strings to select your filters 
played_as = 'ALL'
my_rating_lowerbound = 'ALL'
my_rating_upperbound = 'ALL'
opponent  = 'ALL'
opponent_rating_lowerbound = 'ALL'
opponent_rating_upperbound = 'ALL'
year = 'ALL'
month = 'ALL'
result = 'ALL'
game_ending = 'ALL'
game_type = 'ALL'
time_controls = 'ALL'
opening = 'ALL'


# Calling function to print filtered dataframe (do not edit this)
filtered_results(played_as, my_rating_lowerbound, my_rating_upperbound, opponent, opponent_rating_lowerbound, opponent_rating_upperbound,
                 year, month, result, game_ending, game_type, time_controls, opening)

<br> 

### Display your record for the games you selected
>#### You can change the variable in the square brackets if you want to see value counts for other columns

In [ ]:
filtered_df['Result'].value_counts()

<br> 

### Display the count of the top ten opening variations for the games you selected and your win percentages

In [ ]:
grouped = filtered_df.groupby('Opening')['Result'].value_counts().unstack(fill_value=0)
grouped['win_percentage'] = (grouped['win'] / grouped.sum(axis=1)) * 100
grouped['win_percentage'] = grouped['win_percentage'].round(1)

category_counts = filtered_df['Opening'].value_counts()
grouped['count'] = category_counts
grouped = grouped.loc[category_counts.index]
grouped_reset = grouped.reset_index()
grouped_reset = grouped_reset[['Opening', 'count', 'win_percentage']]

# Change the number 10 in the below line if you want to see more than 10 results
grouped_reset = grouped_reset.head(10)

print(grouped_reset.to_string(index=False))

<br> 

### Save the filtered dataframe as an excel file

In [ ]:
filtered_df.to_excel(f'{username}_chess_games.xlsx', index=False)